In [ ]:
# just some thinking about trying to implement computational neuroscience operations from the dayan and abbot book
# which should be really fascinating, and deriving correct algorithms for these things!


# may not be the most efficient representation, but having this as a type is probably worthwhile
mutable struct SpikeTrain
    spikes::Array{Spike}
end

mutable struct Spike
    time
    val::Number
end

# do custom sorting via a custom comparator
sort!(s::SpikeTrain) = SpikeTrain(sort!(s.spikes, by=first))
sort!(s::Array{Spike}) = sort!(s, by=first)

# assumes sorted spiketrains which is absolutely required!
spike_count_rate(s::SpikeTrain) = length(s.spikes) / (last(s.spikes).time - first(s.spikes).time)

# the average number of spike counts observed over trials
spike_count_average(s::Array{SpikeTrain}) = mean([spike_count_rate(e) for e in s])

# compute simple bin averaging over spiketrains to get the time-dependent firing rate r(t)
function firing_rate_binned(s::SpikeTrain, num_bins::Int)
    dur = (last(s).time - first(s).time) / num_bins
    rates = []
    cur_bin_num::Int = 1
    spikes_per_bin = 0
    for spike in s
        t = spike.time
        if t > dur * cur_bin_num # check if flows over to the next bin. if not add it to the current bin
            push!(rates, spikes_per_bin / dur) # push the previous bin onto rates
            spikes_per_bin = 1
            cur_bin_num +=1
        else
            spikes_per_bin +=1
        end
    end
    # at the end push the final numbers
    push!(rates, spikes_per_bin)
    return rates
end
# do this the really inefficient double loop way. It'll be O(n^2). There is undoubtedly a better way
function firing_rate_window(s::SpikeTrain,window_func, window_size, window_resolution)
    step = (last(s).time - first(s).time) / window_resolution
    windows = []
    wind = 0
    for i in 1:window_resolution
        p = i * step
        for spike in s
            if s.time <= p + (window_size/2) && s.time >= p - (window_size/2) # I think this is the right conditional
                wind += window_func(s.time, p)
            end
        end
        push!(windows, wind)
    end
    return windows
end

function firing_rate_window_spikes(s::SpikeTrain, window_func, window_size)
    windows = []
    wind = 0
    for spike in s
        t = s.time
        for s2 in s
            t2 = s2.time
            if t <= t2 + (window_size/2) && t.time >= t2 - (window_size/2) # I think this is the right conditional
                wind += window_func(s.time, p)
            end
        end
        wind = 0
        push!(windows, wind)
    end
    return windows
end

# define some window functions!
rectangle(t,p) = 1 # always 1 as it just coutns the number in the window
causal_rectangle(t,p) = int(t < p) # only responds if the spike is less than the time of the point - i.e. each point can onl ylook into the past
gaussian(t,p,s) = exp(-(t-p)^2/2*s)
function causal_gaussian(t,p,s)
    if t < p
        return gaussian(t,p,s)
    end
    return 0
end
function causal_alpha(t,p,a)
    if t < p
        return a^2 * p * exp(- (a * p))
    end
    return 0
end
    
# now lets define the average stimulus - i.e. the spike triggered stmiulus values - the val value in the spike trai nis the stimulus
# immedaitely beforehand i.e.
# this works with any type that can be summed as the stimulus!
spike_triggered_average(s::SpikeTrain) = sum([spike.val for spike in s.spikes]) / length(s.spikes)
spike_triggered_average(s::Array{SpikeTrain}) = mean([spike_triggered_average(st) for st in s])
    
function autocorrelation(s::SpikeTrain, window_size, window_func)
    windows = firing_rate_window_spikes(s, window_size, window_func) # compute the instanteous windowed firing rates
    cors = []
    l = length(s.spikes)
    for i in 1:l
        push!(cors, s.spikes[i].val * windows[i] / l )
    end
    return mean(cors)
end
# hopefully this is sufficient but who knows?
# and the mean function here! using multiple dispatch
autocorrelation(s::Array{SpikeTrain}) = mean([autocorrelation(st) for st in s])

# the fano factor is the ratio of the mean and variance of the spike train firing rates!
# for a homogenous poisson process, the fano factor should be 1 since the mean and variance of a poisson are the same!
FanoFactor(s::SpikeTrain) = mean(firing_rate_window_spikes(s)) / var(firing_rate_window_spikes)
fano(s::SpikeTrain) = FanoFactor(s)

# okay, so the key for tomorrow is to continue on with the neural encoding and the poisson processes that I had today
# and get that to work! because the rest of today, at least until 5/6ish I should/need to be focusd onthe hemispheric topology
# and trying out various models to get that to work, which should be interesting, so who knwos? I honestl don't know and it is funny!
#but experiment with various models wrt that and try to get some fun results with that which should be requisite as that cuold/potentially might be
# a further interesting paper to get my name on, which will not be bad 2/3 first name papers if I canget saliency sorted
# and then a second name which si nto the end of the world, then a whole bunch of PP stuff!
    
        
        
        
    

